In [ ]:
import numpy as np
import torch
import pygame
import random
from torch import nn
from torch.distributions import Categorical
import torch.nn.functional as F
import copy
from collections import deque
import tqdm

colors = [
    (0, 0, 0),
    (120, 37, 179),
    (100, 179, 179),
    (80, 34, 22),
    (80, 134, 22),
    (180, 34, 22),
    (180, 34, 122),
]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
class Event():
    type = None
    key = None

    def __init__(self, type, key):
        self.type = type
        self.key = key

In [ ]:
class Figure:
    x = 0
    y = 0

    figures = [
        [[1, 5, 9, 13], [4, 5, 6, 7]],
        [[4, 5, 9, 10], [2, 6, 5, 9]],
        [[6, 7, 9, 10], [1, 5, 6, 10]],
        [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]],
        [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]],
        [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]],
        [[1, 2, 5, 6]],
    ]

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = random.randint(0, len(self.figures) - 1)
        self.color = 1
        self.rotation = 0

    def image(self):
        return self.figures[self.type][self.rotation]

    def rotate(self):
        self.rotation = (self.rotation + 1) % len(self.figures[self.type])


In [ ]:
class Tetris:
    def __init__(self, height, width):
        self.level = 2
        self.score = 0
        self.field = []
        self.height = 0
        self.width = 0
        self.x = 100
        self.y = 60
        self.zoom = 20
        self.figure = None
    
        self.height = height
        self.width = width
        self.field = []
        self.score = 0
        self.done = False
        for i in range(height):
            new_line = []
            for j in range(width):
                new_line.append(0)
            self.field.append(new_line)
            
    def reset(self):
        self.field = []
        self.score = 0
        self.done = False
        for i in range(self.height):
            new_line = []
            for j in range(self.width):
                new_line.append(0)
            self.field.append(new_line)

    def new_figure(self):
        self.figure = Figure(3, 0)

    def intersects(self, figure):
        intersection = False
        for i in range(4):
            for j in range(4):
                if i * 4 + j in figure.image():
                    if i + figure.y > self.height - 1 or \
                            j + figure.x > self.width - 1 or \
                            j + figure.x < 0 or \
                            self.field[i + figure.y][j + figure.x] > 0:
                        intersection = True
        return intersection
    
    def step(self, figure):  
        state = copy.deepcopy(self.field)
        for i in range(4):
            for j in range(4):
                if i * 4 + j in figure.image():
                    state[i + figure.y][j + figure.x] = figure.color
        return state

    def break_lines(self):
        lines = 0
        for i in range(0, self.height):
            zeros = 0
            for j in range(self.width):
                if self.field[i][j] == 0:
                    zeros += 1
            if zeros == 0:
                lines += 1
                del self.field[i]
                self.field.insert(0, [0]*self.width)
        self.score += lines ** 2

    def go_space(self):
        while not self.intersects(self.figure):
            self.figure.y += 1
        self.figure.y -= 1
        self.freeze()

    def go_down(self):
        self.figure.y += 1
        if self.intersects(self.figure):
            self.figure.y -= 1
            self.freeze()

    def freeze(self):
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    self.field[i + self.figure.y][j + self.figure.x] = self.figure.color
        self.break_lines()
        self.new_figure()
        self.done =  self.intersects(self.figure)

    def go_side(self, dx):
        old_x = self.figure.x
        self.figure.x += dx
        intersects = self.intersects(self.figure)
        if intersects:
            self.figure.x = old_x
        return intersects

    def rotate(self):
        old_rotation = self.figure.rotation
        self.figure.rotate()
        if self.intersects(self.figure):
            self.figure.rotation = old_rotation


In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.weights, self.biases = self._init_weights()

    def _init_weights(self):
        weights = []
        biases = []
        layer_sizes = [self.input_size] + self.hidden_layers + [self.output_size]

        for i in range(len(layer_sizes) - 1):
            weight_shape = (layer_sizes[i], layer_sizes[i+1])
            bias_shape = (layer_sizes[i+1],)
            weights.append(np.random.normal(size=weight_shape))
            biases.append(np.random.normal(size=bias_shape))

        return weights, biases

    def forward(self, x):
        hidden_layer = x
        for weight, bias in zip(self.weights[:-1], self.biases[:-1]):
            hidden_layer = np.dot(hidden_layer, weight)
            hidden_layer = np.maximum(0, hidden_layer)  # ReLU activation function
        output_layer = np.dot(hidden_layer, self.weights[-1]) + self.biases[-1]
        return output_layer

    def mutate(self, mutation_rate):
        for i, weight in enumerate(self.weights):
            if np.random.uniform() < mutation_rate:
                self.weights[i] += np.random.normal(size=weight.shape)
        for i, bias in enumerate(self.biases):
            if np.random.uniform() < mutation_rate:
                self.biases[i] += np.random.normal(size=bias.shape)
    def printW(self):
        print(self.weights)

    def printB(self):
        print(self.biases)

    def saveWeights(self):
        return (self.weights, self.biases)
    def copy_from(self, other):
        self.weights = [np.copy(w) for w in other.weights]
        self.biases = [np.copy(b) for b in other.biases]
        
    def parameters(self):
        return self.weights + self.biases

    def zero_grad(self):
        for i in range(len(self.weights)):
            self.weights[i].fill(0)
        for i in range(len(self.biases)):
            self.biases[i].fill(0)

    def apply_gradient(self, gradients, learning_rate):
        for i, grad in enumerate(gradients):
            if i < len(self.weights):
                self.weights[i] -= learning_rate * grad
            else:
                self.biases[i - len(self.weights)] -= learning_rate * grad
                
    def load_weights(self, file_path):
        state_dict = torch.load(file_path)
        for i in range(len(self.weights)):
            self.weights[i] = state_dict[f"layer_{i}_weights"]
            self.biases[i] = state_dict[f"layer_{i}_bias"]




In [ ]:
feature_len = 4
input_size = feature_len
hidden_layers = [32, 16]  
output_size = 1

neural_net = NeuralNetwork(input_size, hidden_layers, output_size)

In [ ]:
def get_state_properties(state):
    aggregate_height = 0
    complete_lines = 0
    holes = 0
    bumpiness = 0
    height_board = len(state)
    width_board = len(state[0])

    for col in range(width_board):
        column_height = 0
        hole_flag = False
        for row in range(height_board):
            if state[row][col]:
                if not hole_flag:
                    column_height = height_board - row
                    hole_flag = True
                aggregate_height += column_height
            else:
                if hole_flag:
                    holes += 1

    for row in range(height_board):
        if all(state[row][col] for col in range(width_board)):
            complete_lines += 1

    column_heights = [height_board - row for row in range(height_board) for col in range(width_board) if state[row][col]]

    for i in range(1, len(column_heights)):
        bumpiness += abs(column_heights[i] - column_heights[i - 1])

    features = np.array([aggregate_height, complete_lines, holes, bumpiness], dtype=np.float32)

    return features



def simulate(t, neural_net):
    fig = Figure(3, 0)
    fig.type = t.figure.type
    fig.color = t.figure.color
    opt = float("-inf")
    opt_rotation, opt_x = 0, fig.x
    if t.intersects(fig):
        return opt_rotation, opt_x
    fig.x = -3
    for i in range(t.width + 3):
        for j in range(len(fig.figures[fig.type])):
            if not t.intersects(fig):
                while not t.intersects(fig):
                    fig.y += 1
                fig.y-=1
                possible_state = t.step(fig)
                features  = get_state_properties(possible_state)
                score_prediction = neural_net.forward(features)
                
                if score_prediction > opt:
                    opt = score_prediction
                    opt_rotation = fig.rotation
                    opt_x = fig.x
                fig.y = 0
            fig.rotate()
        fig.x += 1
    return opt_rotation, opt_x

def run_ai(t, neural_net):
    rotation, x = simulate(t, neural_net)
    if t.figure.rotation != rotation:
        t.figure.rotation = rotation
    elif t.figure.x != x:
        t.figure.x = x
    else:
        t.go_space()
        return [Event(pygame.KEYDOWN, pygame.K_SPACE)]
    return []


In [ ]:
def evaluate_neural_net(neural_net, num_games=5):
    total_score = 0
    for _ in range(num_games):
        t = Tetris(height=20, width=10)
        t.reset()
        t.new_figure()
        while not t.done:
            run_ai(t, neural_net)  # Pass the neural network to the run_ai function
            t.go_space()
        total_score += t.score
    return total_score

In [ ]:
import time
start_time = time.time()

population_size = 200
num_generations = 50
mutation_rate = 0.2

population = [NeuralNetwork(input_size, hidden_layers, output_size) for _ in range(population_size)]

for generation in range(num_generations):
    total_scores = []

    # Calculate the total score for each neural network in the population
    for neural_net in population:
        total_score = evaluate_neural_net(neural_net)
        total_scores.append(total_score)
        # print('totalscore: ', total_score)

    # Select the best performing neural networks
    best_indices = np.argsort(total_scores)[-population_size // 2:]  # Keep the top 50% performers
    best_networks = [population[i] for i in best_indices]

    # Step 3: Create a new generation of neural networks by mutating the best performing networks
    new_population = []
    for best_net in best_networks:
        offspring = copy.deepcopy(best_net)
        offspring.mutate(mutation_rate)
        new_population.append(offspring)
    population = new_population
    # print("generation: ", generation)
    

# Select the best neural network after all generations
best_neural_net = population[np.argmax(total_scores)]
# print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
FILE = "modelNeat-200-50.pth"
torch.save(best_neural_net, FILE)

In [ ]:
file_path = "modelNeat300-100.pth"
best_neural_net = torch.load(file_path)

In [ ]:
def simulate_with_best_nn(t, best_nn):
    fig = Figure(3, 0)
    fig.type = t.figure.type
    fig.color = t.figure.color
    opt = float("-inf")
    opt_rotation, opt_x = 0, fig.x
    if t.intersects(fig):
        return opt_rotation, opt_x
    fig.x = -3
    for i in range(t.width + 3):
        for j in range(len(fig.figures[fig.type])):
            if not t.intersects(fig):
                while not t.intersects(fig):
                    fig.y += 1
                fig.y -= 1
                possible_state = t.step(fig)
                feature = get_state_properties(possible_state)
                score = best_nn.forward(feature).item()
                
                if score > opt:
                    opt = score
                    opt_rotation = fig.rotation
                    opt_x = fig.x
                fig.y = 0
            fig.rotate()
        fig.x += 1
    return opt_rotation, opt_x


def run_ai_with_best_nn(t, best_neural_net):
    rotation, x = simulate_with_best_nn(t, best_neural_net)
    if t.figure.rotation != rotation:
        t.figure.rotation = rotation
    elif t.figure.x != x:
        t.figure.x = x
    else:
        return [Event(pygame.KEYDOWN, pygame.K_SPACE)]
    return []


In [ ]:
# Initialize the game engine
pygame.init()

# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)

size = (400, 500)
screen = pygame.display.set_mode(size)

pygame.display.set_caption("Tetris")

# Loop until the user clicks the close button.
done = False
clock = pygame.time.Clock()
fps = 10
game = Tetris(20, 10)
counter = 0

pressing_down = False

while not done:
    if game.figure is None:
        game.new_figure()
    counter += 1
    if counter > 100000:
        counter = 0

    if counter % (fps // game.level // 2) == 0 or pressing_down:
        if not game.done:
            game.go_down()

    for event in list(pygame.event.get()) + run_ai_with_best_nn(game, best_neural_net):
        if event.type == pygame.QUIT:
            done = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                game.rotate()
            if event.key == pygame.K_DOWN:
                pressing_down = True
            if event.key == pygame.K_LEFT:
                game.go_side(-1)
            if event.key == pygame.K_RIGHT:
                game.go_side(1)
            if event.key == pygame.K_SPACE:
                game.go_space()
            if event.key == pygame.K_ESCAPE:
                game.__init__(20, 10)

    if event.type == pygame.KEYUP:
            if event.key == pygame.K_DOWN:
                pressing_down = False

    screen.fill(WHITE)

    for i in range(game.height):
        for j in range(game.width):
            pygame.draw.rect(screen, GRAY, [game.x + game.zoom * j, game.y + game.zoom * i, game.zoom, game.zoom], 1)
            if game.field[i][j] > 0:
                pygame.draw.rect(screen, colors[game.field[i][j]],
                                 [game.x + game.zoom * j + 1, game.y + game.zoom * i + 1, game.zoom - 2, game.zoom - 1])
    
    if game.figure is not None:
        for i in range(4):
            for j in range(4):
                p = i * 4 + j
                if p in game.figure.image():
                    pygame.draw.rect(screen, colors[game.figure.color],
                                     [game.x + game.zoom * (j + game.figure.x) + 1,
                                      game.y + game.zoom * (i + game.figure.y) + 1,
                                      game.zoom - 2, game.zoom - 2])

    font = pygame.font.SysFont('Calibri', 25, True, False)
    font1 = pygame.font.SysFont('Calibri', 65, True, False)
    text = font.render("Score: " + str(game.score), True, BLACK)
    text_game_over = font1.render("Game Over", True, (255, 125, 0))
    text_game_over1 = font1.render("Press ESC", True, (255, 215, 0))

    screen.blit(text, [0, 0])
    if game.done:
        screen.blit(text_game_over, [20, 200])
        screen.blit(text_game_over1, [25, 265])

    pygame.display.flip()
    clock.tick(fps)

pygame.quit()